## Many to many relationship
- 한 테이블의 0개 이상의 레코드가 다른 테이블의 0개 이상의 레코드와 관련된 경우
- 양쪽 모두에서 서로 N:1 관계를 가짐

### 데이터 모델링
- 주어진 개념으로부터 논리적인 데이터 모델을 구성하는 작업
- 물리적인 데이터베이스 모델로 만들어 고객의 요구에 따라 특정 정보 시스템의 데이터베이스에 반영하는 작업

* 시작하기 전 용어정리
  - target model: 관계 필드를 가지지 않은 모델
  - source model: 관계 필드를 가진 모델

### N:1의 한계
- 의사와 환자간 예약 시스템을 구현
- 지금까지 배운 N:1 관계를 생각해 한 명의 의사에게 여러 환자가 예약할 수 있다고 모델 관계를 설정
- 한 환자가 여러 의사에게 방문하려 하면 객체를 하나 더 만들어서 예약을 진행해야 함
- 예약 테이블이라면 두 번 객체가 만들어져도 되지만, 환자 테이블이므로 객체가 하나인것이 더 어울림
- 따라서 예약 테이블을 따로 만들자

#### 중개 모델
- 환자 모델의 외래 키를 삭제하고 별도의 예약 모델을 새로 작성
- 예약 모델은 의사와 환자에게 각각 N:1의 관계를 가짐
- 중개 모델을 통해 역참조로 환자가 예약한 의사들, 의사에 예약한 환자들 조회 가능
- 다만 이렇게 할시 중개모델인 reservation이 예약을 만듬. 실제로는 환자나 의사가 만들어야 하므로 어색한 구조임.

### Django ManyToManyField
- 환자 모델에 Django ManyToManyField 작성
- 환자 모델이든 의사 모델이든 상관없이 작성해도 됨. 단, 참조와 역참조 주의!
- migrate 할시 알아서 중개테이블 생성됨
- 의자와 환자를 생성하고 예약 자체를 생성할 수 있음

#### 환자가 주체
- patient1.doctors.add(doctor1) : 환자1이 의사1에게 예약
- patient1.doctors.all() : 환자1이 예약한 의사목록 확인
- doctor1.patient_set.all() : 자신의 예약된 환자목록 확인

#### 의사가 주체
- doctor1.patient_set.add(patient2) : 의사1이 환자2를 예약
- doctor1.patient_set.all() : 의사1 자신의 예약 환자목록 확인
- patient1.doctors.all() : 환자 1이 예약한 의사목록 확인

#### 예약 취소하기
- doctor1.patient_set.remove(patient1)
- patient1.doctors.remove(doctor1)

### related_name argument
- ManyToManyField(Doctor, related_name='patients')
- target 모델이 source모델을 참조할때(역참조) 사용할 manager name
- ForeignKey()의 related_name과 동일

### through argument
- 중개 모델을 직접 작성할수 없을까
- 더 많은 데이터를 중개테이블에 추가하고 싶을때 through 옵션을 사용하여 사용하려는 중개테이블을 나타내는 django 모델을 지정할 수 있음
- 가장 일반적인 용도는 중개테이블에 추가 데이터를 사용
- 이렇게 하면 reservation 클래스를 통해 예약 가능
- 단, patient나 doctor를 통해 예약을 생성할때 through_defaults={'symptom': 'flu'} 와 같은 형태로 추가적인 데이터를 입력해주어야 함

### 정리
- M:N 관계로 맺어진 두 테이블에는 변화가 없음
- ManyToManyField는 중개테이블을 자동 생성하며 두 모델 어디에 위치해도 상관 없으나 참조와 역참조를 주의해야 한다.
- N:1은 종속의 관계였지만 M:N은 의사에게 진찰받는 환자, 환자를 진찰하는 의사의 두가지 형태로 모두 표현이 가능한 것

### ManyToManyField
- ManyToManyField(to, **options)
- 하나의 필수인자(M:N관꼐로 설정할 모델 클래스)가 필요
- add, remove, create, clear 등을 사용하여 개체를 추가, 제거, 생성 가능\
- db_table arguments를 사용하여 중개 테이블의 이름 변경 가능
- realted_name
  - 역참조시 사용할 manager name
- through
  - 중개테이블 직접 작성시 사용
- symmetrical
  - 기본값: True
  - ManyToManyField가 동일한 모델(on self)을 가리키는 정의에서만 사용
  - True일시 한 쪽의 관계가 추가되면 (1, 2가 추가되면) 반대도 성립. (2, 1도 추가)
  - 예) 팔로우(한쪽이 팔로우해도 맞팔이 아닌 일방적인 팔로우임)
  - _set 매니저를 추가하지 않음


#### Related Manager
- N:1, M:N관계에서 사용 가능한 문맥
- 역참조시에 상요할 수 있는 manager 생성
- 같은 이름의 메서드여도 각 관계(N:1, M:N)에 따라 다르게 사용 및 동작
- add, remove, create, clear, set 등
- add
  - 지정된 객체를 관련 객체 집합에 추가
  - 이미 존재하는 관계에 사용하면 관계가 복제되지 않음
- remove
  - 내부적으로 QuerySet.delete()를 사용하여 관계 삭제